<a href="https://colab.research.google.com/github/harryypham/MyMLPractice/blob/main/random/mt_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency 

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
from datasets import load_dataset

books = load_dataset('opus_books', 'en-fr')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [4]:
print('type of dataset:', type(books))
print('raw dataset:', books)
processed_data = books['train'].train_test_split(test_size=0.2)
print(processed_data)


type of dataset: <class 'datasets.dataset_dict.DatasetDict'>
raw dataset: DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})
DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 25417
    })
})


In [5]:
from transformers import AutoTokenizer

checkpoint = 'google-t5/t5-small'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [68]:
source_lang = 'en'
target_lang = 'fr'
prefix = 'translate English to Vietnamese: '

def preprocess_func(examples):
  inputs = [prefix + example[source_lang] for example in examples['translation']]
  targets = [example[target_lang] for example in examples['translation']]
  model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
  return model_inputs

In [63]:
tokenized_books = processed_data.map(preprocess_func, batched=True)
print(tokenized_books)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25417
    })
})


In [58]:
print(len(tokenizer.get_vocab().keys()))

32100


In [69]:
print(tokenized_books['train']['id'])

KeyError: "Column id not in the dataset. Current columns in the dataset: ['input_ids', 'attention_mask', 'labels']"

In [70]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
print(type(data_collator))

<class 'transformers.data.data_collator.DataCollatorForSeq2Seq'>


In [12]:
import evaluate

metric = evaluate.load("sacrebleu")

In [71]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [65]:
print(tokenized_books)
# have to remove columns with data types string otherwise collator don't know how to pad
tokenized_books = tokenized_books.remove_columns(['id', 'translation'])
tokenized_books.set_format('torch')
print(tokenized_books)

DatasetDict({
    train: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['id', 'translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25417
    })
})
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 101668
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25417
    })
})


In [66]:
from torch.utils.data import DataLoader
train_data = tokenized_books['train']
train_loader = DataLoader(tokenized_books['train'], shuffle=True, batch_size=4, collate_fn=data_collator, pin_memory=True)
val_loader = DataLoader(tokenized_books['test'], batch_size=4, collate_fn=data_collator, pin_memory=True)

In [72]:
batch = next(iter(train_loader))
print('input shape:', batch['input_ids'].shape)
print('attention mask shape:', batch['attention_mask'].shape)
print('target shape:', batch['labels'].shape)

input shape: torch.Size([4, 76])
attention mask shape: torch.Size([4, 76])
target shape: torch.Size([4, 112])


In [80]:
from transformers import AutoModelForSeq2SeqLM
checkpoint = "google/mt5-base"
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint, legacy=False)
# output = model(**batch)
# print(output.logits.shape)
# target_ids = torch.softmax()
text = "translate English to French: What is the best restaurant in Sydney?"
inputs = tokenizer(text, return_tensors="pt").input_ids
print(inputs)

outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[37194,  5413,   288, 21273,   267,  5126,   339,   287,  1920, 13306,
           281, 40654,   291,     1]])
<extra_id_0> (see which one)


In [51]:
import torch
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
optimizer = AdamW(model.parameters(), lr=2e-5)
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type='cuda', dtype=ptdtype)
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=50, num_training_steps=num_training_steps
)


/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [ ]:
print('compiling model...')
model = torch.compile(model)
model.to(device)

print(f'training with {dtype}')
pbar = tqdm(range(num_training_steps))
model.train()

for epoch in range(num_epochs):
  for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with ctx:
          outputs = model(**batch)
          loss = outputs.loss

        scaler.scale(loss).backward()

        lr_scheduler.step()
        scaler.step(optimizer)
        scaler.update()

        optimizer.zero_grad()
        pbar.update(1)

In [83]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"
article = "UN Chief Says There Is No Military Solution in Syria"
inputs = tokenizer(article, return_tensors="pt")


tensor([[     2, 250024,  19167,  29004,   2494,  25332, 140857,   8735,   2872,
            687,    524,   8652,   6800,  29225,   2550,   2059,  51712,      2]])


['Người đứng đầu Liên Hiệp Quốc nói không có giải pháp quân sự ở Syria']

In [84]:
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["vi_VN"], output_scores=True)
print(translated_tokens)
tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

tensor([[     2, 250024,  19167,  29004,   2494,  25332, 140857,   8735,   2872,
            687,    524,   8652,   6800,  29225,   2550,   2059,  51712,      2]])


['Người đứng đầu Liên Hiệp Quốc nói không có giải pháp quân sự ở Syria']